In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import os
import pickle
from PIL import Image
from keras.utils import np_utils
from scipy import misc

Using TensorFlow backend.


In [ ]:
path = 'dataset/train/Train/'

In [ ]:
images = os.listdir(path)

In [ ]:
images[0]

In [ ]:
Image.open('dataset/train/Train/'+images[3]).resize((32,32))

In [ ]:
train_labels = pd.read_csv('dataset/train/train.csv')

In [ ]:
train_labels.head()

In [ ]:
# outputs = list((train_labels['Class']).unique())

In [ ]:
# (train_labels[train_labels['ID']==images[0]]['Class'])

In [ ]:
class data_Label_Combiner:

    def Data_combiner(self,images, labels, path):
        x_train=[]
        y_train=[]
        for each in images:
            x_train.append(np.asarray(Image.open(path+each).resize((64,64)), dtype='int32'))
#             y_train.append(np.asarray(binarizer.transform(labels[labels['ID']==each]['Class']),dtype='float32'))
            y_train.append(np.asarray(labels[labels['ID']==each]['Class']))
        return x_train, y_train


In [ ]:
dlc = data_Label_Combiner()

In [ ]:
x_train, y_trainer = dlc.Data_combiner(images,train_labels,path)

In [ ]:
y_train= np.array(y_trainer)

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit(y_train)

In [ ]:
y_train = le.transform(y_train)

In [ ]:
y_train = np_utils.to_categorical(y_train)

In [ ]:
x_train=np.array(x_train)

In [ ]:
x_train, x_cross_val, y_train, y_cross_val = train_test_split(x_train,y_train, test_size = 0.1)

In [ ]:
clasifier = Sequential()

In [ ]:
clasifier.add(Convolution2D(32,3,3, input_shape=(64,64,3), activation='relu'))
clasifier.add(MaxPooling2D(pool_size=(2,2)))
clasifier.add(Convolution2D(32,3,3, activation='relu'))
clasifier.add(MaxPooling2D(pool_size=(2,2)))
clasifier.add(Flatten())

In [ ]:
clasifier.add(Dense(activation='relu', output_dim = 100))
# clasifier.add(Dense(activation='relu', output_dim = 128))
clasifier.add(Dense(output_dim = 3, activation='softmax'))


In [ ]:
clasifier.compile(metrics=['accuracy'], loss='binary_crossentropy', optimizer='adam' )

In [ ]:
# train_generator = ImageDataGenerator(rotation_range=0.2,zoom_range=0.3, shear_range=0.3, vertical_flip=True, rescale=1.0/255)

train_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    zoom_range=0.3,
    shear_range=0.3,
    rescale=1.0/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

val_generator = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator.fit(x_train)
val_generator.fit(x_cross_val)

In [ ]:
# x = np.array([train_generator.apply_transform(sample,transform_parameters={'theta':30,'tx':0.2,'ty':0.3,'shear':0.6,'rescale':1.0/256,'zoom_range':0.3,'shear_range':0.5,'horizontal_flip':True}) for sample in x_train])

In [ ]:
# type(x)

In [ ]:
# x_c = np.array([val_generator.apply_transform(sample,transform_parameters={'theta':30,'tx':0.2,'ty':0.3,'shear':0.6}) for sample in x_cross_val])

In [ ]:
# x_train.shape

In [ ]:
# for i,j in train_generator.flow(x_train,y_train):
#     print (i,j,'\n\n')
# np.array(x_train).shape, np.array(y_train).shape

In [ ]:
# for e in range(10):
# #     print('Epoch', e)
#     batches = 0
#     for x_batch, y_batch in train_generator.flow(x, y_train, batch_size=32):
#         clasifier.fit(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break

In [ ]:
clasifier.fit_generator(train_generator.flow(x_train,y_train,batch_size=32), steps_per_epoch=len(x_train), epochs=10,validation_data=val_generator.flow(x_cross_val,y_cross_val,batch_size=32),validation_steps=len(x_cross_val))

In [ ]:
# clasifier.fit(x_train,y_train,steps_per_epoch=len(x_train) ,epochs=10,verbose=1,validation_data=(x_cross_val,y_cross_val),validation_steps=len(x_cross_val))

In [ ]:
clasifier.evaluate(x_cross_val,y_cross_val)

In [ ]:
y_pred = clasifier.predict_classes(x_cross_val)

In [ ]:
y_pred[0]

In [ ]:
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
confusion_matrix(y_trainer,y_pred)

In [ ]:
y_cross_val

In [ ]:
with open('pickle/model.pkl','wb') as f:
    pickle.dump(dlc,f)
    pickle.dump(le,f)
    pickle.dump(clasifier,f)